In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install torch
!pip install transformers
!pip install pydub
!pip install edge-tts
!pip install nest_asyncio
!pip install ipython

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-wu90bqu5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-wu90bqu5
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)


In [2]:
from IPython.display import Audio, display

# Play the audio file
def play_audio(file_path):
    display(Audio(file_path, autoplay=True))

# Example usage
input_file = "/content/voice_msg/llm_input_1.m4a"
play_audio(input_file)

In [3]:
import whisper
import torchaudio

# Load the Whisper model
model = whisper.load_model("base")

# Load your .m4a audio file
waveform, sample_rate = torchaudio.load("/content/voice_msg/llm_input_1.m4a")

# Resample the audio to 16000 Hz if it's not already
resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
waveform = resampler(waveform)

# Convert waveform to mono (Whisper expects mono audio)
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

vad = torchaudio.transforms.Vad(sample_rate=16000)

# Apply VAD to detect voice activity and ignore silence
waveform_vad = vad(waveform)

# Convert waveform tensor to numpy array for Whisper
waveform = waveform_vad.squeeze().numpy()

# Transcribe the audio waveform
result = model.transcribe(waveform)




100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 82.9MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [4]:
# Print the transcription result
print(result["text"])

 What is machine learning?


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("gokaygokay/Lamini-Prompt-Enchance-Long")
model = AutoModelForSeq2SeqLM.from_pretrained("gokaygokay/Lamini-Prompt-Enchance-Long")

# Function to generate response using the LLM
def generate_response(input_text, max_length=500):
    # Tokenize the input text
    inputs = tokenizer.encode(input_text, return_tensors="pt")

    # Generate response from the model
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1)

    # Decode the generated tokens to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Split the response into sentences
    sentences = response.split('.')

    # Return only the first two sentences
    return '. '.join(sentences[:2]).strip() + '.'




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [6]:
input_text = result["text"]
response_text = generate_response(input_text)
print("LLM Response:", response_text)

LLM Response: A machine learning technique is a subset of artificial intelligence that focuses on creating algorithms that can learn from data and make predictions or decisions based on that learning..


In [11]:
import edge_tts
import asyncio
import nest_asyncio

# Apply nest_asyncio to avoid RuntimeError
nest_asyncio.apply()

async def text_to_speech(text, output_file, voice="en-US-AriaNeural"):
  # voice = "en-US-GuyNeural"  # Male voice
  # voice = "en-US-AriaNeural"  # Female voice
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(output_file)
    print(f"Audio saved to {output_file}")

# Example usage
text = response_text
output_file = "output_audio.mp3"

# Run the async function in the current event loop
await text_to_speech(text, output_file)



Audio saved to output_audio.mp3


In [12]:
from IPython.display import Audio, display

# Play the audio file
def play_audio(file_path):
    display(Audio(file_path, autoplay=True))

# Example usage
output_file = "output_audio.mp3"
play_audio(output_file)
